In [1]:
from PIL import Image, ImageChops, ImageEnhance
import sys, os.path
import numpy as np
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, load_model
from keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout, Activation, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import utils

np.random.seed(2)
tf.random.set_seed(2)

In [2]:
image_path_ela="D:/Casia/CASIA1/Ela"
image_path_resaved="D:/Casia/CASIA1/Ela/resaved"
image_path_augmented_au="D:/Casia/CASIA1/Augment/Au"
image_path_augmented_au="D:/Casia/CASIA1/Augment/Sp"

def ela(filename):
    file_name = filename.split("\\")[-1]
    resaved = f"{image_path_resaved}/{file_name}" + '.resaved.jpg'
    ela = f"{image_path_ela}/{file_name}"  + '.ela.png'
    im = Image.open(filename).convert('RGB')

    im.save(resaved, 'JPEG', quality=95)
    resaved_im = Image.open(resaved)

    ela_im = ImageChops.difference(im, resaved_im)
    extrema = ela_im.getextrema()
    max_diff = max([ex[1] for ex in extrema])
    scale = 255.0/max_diff

    ela_im = ImageEnhance.Brightness(ela_im).enhance(scale)

    ela_im.save(ela)
    return ela_im


In [5]:
datagen_train = ImageDataGenerator(
      rescale=1./255,
      rotation_range=180,
      width_shift_range=0.1,
      height_shift_range=0.1,
      shear_range=0.1,
      zoom_range=[0.9, 1.5],
      horizontal_flip=True,
      vertical_flip=True,
      fill_mode='nearest')

In [17]:
generator_train = datagen_train.flow_from_directory(directory="D:\Casia\CASIA1",
                                                    classes=['Au'],
                                                    batch_size=16,
                                                    shuffle=True,
                                                    save_to_dir="D:/Casia/CASIA1/Augment/Au")
generator_train_sp = datagen_train.flow_from_directory(directory="D:\Casia\CASIA1",
                                                    classes=['Sp'],
                                                    batch_size=16,
                                                    shuffle=True,
                                                    save_to_dir="D:/Casia/CASIA1/Augment/Sp")

Found 800 images belonging to 1 classes.
Found 921 images belonging to 1 classes.


In [3]:
def convert_image_to_array(image_path):
    size=(128,128)
    return np.array(ela(image_path).resize(size)).flatten() / 255.0

In [4]:
X=[]
Y=[]

In [5]:
sp_path="D:\Casia\CASIA1\Sp\\"
au_path="D:\Casia\CASIA1\Au\\"

In [6]:
#authentic (1)
for filename in tqdm(os.listdir(au_path), desc='processing images'):
    if filename.endswith('jpg') or filename.endswith('png'):
        f = os.path.join(au_path, filename)
        X.append(convert_image_to_array(f))
        Y.append(1)

        
print(f'Total images: {len(X)}\nTotal labels: {len(Y)}')

#sp (0)
for filename in tqdm(os.listdir(sp_path), desc='processing images'):
    if filename.endswith('jpg') or filename.endswith('png'):
        f = os.path.join(sp_path, filename)
        X.append(convert_image_to_array(f))
        Y.append(0)

        
print(f'Total images: {len(X)}\nTotal labels: {len(Y)}')

processing images:   0%|          | 0/800 [00:00<?, ?it/s]

Total images: 790
Total labels: 790


processing images:   0%|          | 0/921 [00:00<?, ?it/s]

Total images: 1711
Total labels: 1711


In [7]:
X=np.array(X)
Y=np.array(Y)
X = X.reshape(-1, 128, 128, 3)
print(X.shape)
print(Y.shape)

(1711, 128, 128, 3)
(1711,)


In [8]:
x_train, x_, y_train, y_ = train_test_split(X, Y, test_size=0.20, random_state=1)

x_cv, x_test, y_cv, y_test = train_test_split(x_, y_, test_size=0.50, random_state=1)

del x_, y_

print(f"the shape of the training set (input) is: {x_train.shape}")
print(f"the shape of the training set (target) is: {y_train.shape}\n")
print(f"the shape of the cross validation set (input) is: {x_cv.shape}")
print(f"the shape of the cross validation set (target) is: {y_cv.shape}\n")
print(f"the shape of the test set (input) is: {x_test.shape}")
print(f"the shape of the test set (target) is: {y_test.shape}")

the shape of the training set (input) is: (1368, 128, 128, 3)
the shape of the training set (target) is: (1368,)

the shape of the cross validation set (input) is: (171, 128, 128, 3)
the shape of the cross validation set (target) is: (171,)

the shape of the test set (input) is: (172, 128, 128, 3)
the shape of the test set (target) is: (172,)


In [9]:
orig_model = tf.keras.models.load_model('Casia2_model')

In [10]:
orig_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 128, 128, 32)      2432      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 64, 64, 32)       0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 64, 64, 32)        0         
                                                                 
 conv2d_3 (Conv2D)           (None, 64, 64, 32)        25632     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 32, 32, 32)       0         
 2D)                                                             
                                                                 
 dropout_2 (Dropout)         (None, 32, 32, 32)       

In [11]:
correct_test = 0 #correctly predicted test images
total_test = 0   #total test images

for index,image in enumerate(tqdm(x_test,desc="Processing Images : ")):
    image = image.reshape(-1, 128, 128, 3)
    y_pred = orig_model.predict(image)
    y_pred_class = np.round(y_pred)
    total_test += 1
    if y_pred_class == y_test[index]: #if prediction is correct
        correct_test += 1
        
print(f'Total test images: {total_test}\nCorrectly predicted images: {correct_test}\nAccuracy: {correct_test / total_test * 100.0} %')

Processing Images :   0%|          | 0/172 [00:00<?, ?it/s]

1/1 [==============================] - 0s 21ms/step
Total test images: 172
Correctly predicted images: 102
Accuracy: 59.30232558139535 %


In [12]:
model=Sequential()
for layer in orig_model.layers[:-1]:
    model.add(layer)

In [ ]:
model_1.summary()

In [14]:
for layer in model.layers:
    layer.trainable=False

In [16]:
model.add(Dense(units=1, activation='sigmoid', name='output'))

In [19]:
model.compile(optimizer=Adam(0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_accuracy',
                              min_delta=0,
                              patience=5,
                              verbose=0, mode='auto')
epochs=10
history = model.fit(
    x_train, y_train,
    validation_data=(x_cv, y_cv),
    epochs=epochs,
    batch_size=32,
    callbacks=[early_stopping]
)

Epoch 1/10
43/43 [==============================] - 5s 103ms/step - loss: 0.5688 - accuracy: 0.6667 - val_loss: 0.6307 - val_accuracy: 0.6199
Epoch 2/10
43/43 [==============================] - 4s 102ms/step - loss: 0.5653 - accuracy: 0.6798 - val_loss: 0.6331 - val_accuracy: 0.6199
Epoch 3/10
43/43 [==============================] - 4s 102ms/step - loss: 0.5632 - accuracy: 0.6857 - val_loss: 0.6401 - val_accuracy: 0.6023
Epoch 4/10
43/43 [==============================] - 4s 101ms/step - loss: 0.5619 - accuracy: 0.6754 - val_loss: 0.6276 - val_accuracy: 0.6316
Epoch 5/10
43/43 [==============================] - 4s 102ms/step - loss: 0.5672 - accuracy: 0.6754 - val_loss: 0.6257 - val_accuracy: 0.6140
Epoch 6/10
43/43 [==============================] - 4s 102ms/step - loss: 0.5664 - accuracy: 0.6754 - val_loss: 0.6258 - val_accuracy: 0.5965
Epoch 7/10
43/43 [==============================] - 4s 102ms/step - loss: 0.5681 - accuracy: 0.6681 - val_loss: 0.6232 - val_accuracy: 0.6257
Epoch 

In [20]:
correct_test = 0 #correctly predicted test images
total_test = 0   #total test images

for index,image in enumerate(tqdm(x_test,desc="Processing Images : ")):
    image = image.reshape(-1, 128, 128, 3)
    y_pred = model.predict(image)
    y_pred_class = np.round(y_pred)
    total_test += 1
    if y_pred_class == y_test[index]: #if prediction is correct
        correct_test += 1
        
print(f'Total test images: {total_test}\nCorrectly predicted images: {correct_test}\nAccuracy: {correct_test / total_test * 100.0} %')

Processing Images :   0%|          | 0/172 [00:00<?, ?it/s]

1/1 [==============================] - 0s 23ms/step
Total test images: 172
Correctly predicted images: 111
Accuracy: 64.53488372093024 %


In [68]:
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal",
                      input_shape=(128,
                                  128,
                                  3)),
    layers.RandomRotation(0.5),
    layers.RandomZoom(0.5),
  ]
)

In [69]:
model_1=Sequential()
model_1.add(data_augmentation)
for layer in orig_model.layers[:-2]:
    model_1.add(layer)
    
for layer in model_1.layers:
    layer.trainable=False

In [70]:
model_1.add(Dense(256, activation = 'relu'))
model_1.add(Dense(units=1, activation='sigmoid', name='output'))

In [62]:
model_1.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_10 (Sequential)  (None, 128, 128, 3)       0         
                                                                 
 conv2d_2 (Conv2D)           (None, 128, 128, 32)      2432      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 64, 64, 32)       0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 64, 64, 32)        0         
                                                                 
 conv2d_3 (Conv2D)           (None, 64, 64, 32)        25632     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 32, 32, 32)       0         
 2D)                                                 

In [71]:
model_1.compile(optimizer=Adam(0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_accuracy',
                              min_delta=0,
                              patience=15,
                              verbose=0, mode='auto')
epochs=100
history = model_1.fit(
    x_train, y_train,
    validation_data=(x_cv, y_cv),
    epochs=epochs,
    batch_size=32,
)

Epoch 1/100
43/43 [==============================] - 7s 147ms/step - loss: 0.6734 - accuracy: 0.5950 - val_loss: 0.8690 - val_accuracy: 0.5848
Epoch 2/100
43/43 [==============================] - 6s 143ms/step - loss: 0.6385 - accuracy: 0.6316 - val_loss: 0.6781 - val_accuracy: 0.5965
Epoch 3/100
43/43 [==============================] - 6s 142ms/step - loss: 0.6233 - accuracy: 0.6440 - val_loss: 0.6771 - val_accuracy: 0.5906
Epoch 4/100
43/43 [==============================] - 6s 143ms/step - loss: 0.6251 - accuracy: 0.6528 - val_loss: 0.8779 - val_accuracy: 0.5673
Epoch 5/100
43/43 [==============================] - 6s 142ms/step - loss: 0.6211 - accuracy: 0.6550 - val_loss: 0.7764 - val_accuracy: 0.5731
Epoch 6/100
43/43 [==============================] - 6s 143ms/step - loss: 0.6087 - accuracy: 0.6601 - val_loss: 0.7131 - val_accuracy: 0.5848
Epoch 7/100
43/43 [==============================] - 6s 143ms/step - loss: 0.6228 - accuracy: 0.6411 - val_loss: 0.7292 - val_accuracy: 0.5731

Epoch 58/100
43/43 [==============================] - 6s 150ms/step - loss: 0.5587 - accuracy: 0.6937 - val_loss: 1.1946 - val_accuracy: 0.5731
Epoch 59/100
43/43 [==============================] - 6s 149ms/step - loss: 0.5461 - accuracy: 0.7193 - val_loss: 1.3233 - val_accuracy: 0.5789
Epoch 60/100
43/43 [==============================] - 6s 148ms/step - loss: 0.5533 - accuracy: 0.6930 - val_loss: 1.2615 - val_accuracy: 0.5848
Epoch 61/100
43/43 [==============================] - 6s 146ms/step - loss: 0.5658 - accuracy: 0.6923 - val_loss: 1.2904 - val_accuracy: 0.5731
Epoch 62/100
43/43 [==============================] - 6s 146ms/step - loss: 0.5459 - accuracy: 0.7120 - val_loss: 1.0841 - val_accuracy: 0.5497
Epoch 63/100
43/43 [==============================] - 7s 168ms/step - loss: 0.5525 - accuracy: 0.7039 - val_loss: 1.0669 - val_accuracy: 0.5497
Epoch 64/100
43/43 [==============================] - 7s 156ms/step - loss: 0.5484 - accuracy: 0.6966 - val_loss: 1.2715 - val_accuracy:

In [67]:
correct_test = 0 #correctly predicted test images
total_test = 0   #total test images

for index,image in enumerate(tqdm(x_test,desc="Processing Images : ")):
    image = image.reshape(-1, 128, 128, 3)
    y_pred = model_1.predict(image)
    y_pred_class = np.round(y_pred)
    total_test += 1
    if y_pred_class == y_test[index]: #if prediction is correct
        correct_test += 1
        
print(f'Total test images: {total_test}\nCorrectly predicted images: {correct_test}\nAccuracy: {correct_test / total_test * 100.0} %')

Processing Images :   0%|          | 0/172 [00:00<?, ?it/s]

1/1 [==============================] - 0s 22ms/step
Total test images: 172
Correctly predicted images: 98
Accuracy: 56.97674418604651 %


In [35]:
model_2=Sequential()
for layer in orig_model.layers[:-6]:
    model_2.add(layer)
    
for layer in model_2.layers:
    layer.trainable=False

In [48]:
model_2.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 128, 128, 32)      2432      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 64, 64, 32)       0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 64, 64, 32)        0         
                                                                 
 C2 (Conv2D)                 (None, 64, 64, 32)        25632     
                                                                 
 max_pooling_2d_2 (MaxPoolin  (None, 32, 32, 32)       0         
 g2D)                                                            
                                                                 
 d2 (Dropout)                (None, 32, 32, 32)       

In [39]:
model_2.add(Conv2D(32, 5, padding='same', activation='relu', name='C2'))

In [44]:
model_2.add(MaxPool2D(name='max_pooling_2d_2'))

In [45]:
model_2.add(Dropout(0.2, name='d2'))

In [46]:
model_2.add(Flatten())

In [47]:
model_2.add(Dense(256, activation = 'relu'))
model_2.add(Dense(units=1, activation='sigmoid', name='output'))

In [49]:
model_2.compile(optimizer=Adam(0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_accuracy',
                              min_delta=0,
                              patience=5,
                              verbose=0, mode='auto')
epochs=15
history = model_2.fit(
    x_train, y_train,
    validation_data=(x_cv, y_cv),
    epochs=epochs,
    batch_size=32,
    callbacks=[early_stopping]
)

Epoch 1/15
43/43 [==============================] - 11s 258ms/step - loss: 0.7057 - accuracy: 0.5088 - val_loss: 0.6926 - val_accuracy: 0.5263
Epoch 2/15
43/43 [==============================] - 10s 241ms/step - loss: 0.6918 - accuracy: 0.5373 - val_loss: 0.6921 - val_accuracy: 0.5263
Epoch 3/15
43/43 [==============================] - 11s 245ms/step - loss: 0.6909 - accuracy: 0.5380 - val_loss: 0.6923 - val_accuracy: 0.5263
Epoch 4/15
43/43 [==============================] - 11s 251ms/step - loss: 0.6838 - accuracy: 0.5753 - val_loss: 0.6843 - val_accuracy: 0.5906
Epoch 5/15
43/43 [==============================] - 9s 217ms/step - loss: 0.6304 - accuracy: 0.6681 - val_loss: 0.6138 - val_accuracy: 0.7135
Epoch 6/15
43/43 [==============================] - 10s 236ms/step - loss: 0.5397 - accuracy: 0.7376 - val_loss: 0.5746 - val_accuracy: 0.7018
Epoch 7/15
43/43 [==============================] - 10s 243ms/step - loss: 0.4607 - accuracy: 0.7705 - val_loss: 0.5158 - val_accuracy: 0.7310


In [50]:
correct_test = 0 #correctly predicted test images
total_test = 0   #total test images

for index,image in enumerate(tqdm(x_test,desc="Processing Images : ")):
    image = image.reshape(-1, 128, 128, 3)
    y_pred = model_2.predict(image)
    y_pred_class = np.round(y_pred)
    total_test += 1
    if y_pred_class == y_test[index]: #if prediction is correct
        correct_test += 1
        
print(f'Total test images: {total_test}\nCorrectly predicted images: {correct_test}\nAccuracy: {correct_test / total_test * 100.0} %')

Processing Images :   0%|          | 0/172 [00:00<?, ?it/s]

1/1 [==============================] - 0s 24ms/step
Total test images: 172
Correctly predicted images: 126
Accuracy: 73.25581395348837 %
